Porous Media Pressure Drop Calcs

In [1]:
import numpy as np
from fluids import *


In [10]:
PM_OR = 0.049 # PM outer radius [m]
PM_IR = 0.045 # PM inner radius [m]
PM_thick = PM_OR - PM_IR # PM thickness [m]
L = 0.81 # CFE length [m]

f = H2(T=450, P=12.5e6) # Fluid state

# Define operating conditions
mflow = 0.1 # Mass flow rate [kg/s]
rho = f.rho # Fluid density [kg/m^3]
Q = mflow / rho # Volumetric flow rate [m^3/s]
A = 2*np.pi*PM_OR*L # Cross sectional area [m^2]
q = Q / A # Darcian velocity [m/s]
mu = f.mu # Fluid viscosity [Pa * s]

k1 = 5.07926338073061E-13 # for 18% porosity
k2 = np.exp(-1.71588/(k1**0.08093)) # Correlation from Dey et al.

PM_dP = mu*q/k1 + (rho*q**2)/k2 # Pressure drop across PM

print(f"rho = {rho}")
print(f"Q = {Q}")
print(f"A = {A}")
print(f"q = {q}")
print(f"mu = {mu}")
print(f"k1 = {k1}")
print(f"k2 = {k2}")
print(f"dP = {PM_dP * 10**-6}")


rho = 6.389293864533732
Q = 0.01565118182387713
A = 0.2493796248419578
q = 0.06276046743512399
mu = 1.1895048408744165e-05
k1 = 5.07926338073061e-13
k2 = 4.303743771287432e-08
dP = 2.0545392187250497
